<a href="https://colab.research.google.com/github/dung18520632/Deep_Learning-Python-Basics/blob/main/Logistic%20Regression%20with%20a%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

In [2]:
import numpy as np
import h5py
import os      

def load_dataset():
    train_dataset = h5py.File('/content/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('/content/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [69]:
train_X,test_X,train_Y,test_Y,classes=load_dataset()

In [70]:
# Mỗi cột sẽ biểu diễn đặc trưng của 1 ảnh
train_X_flatten=train_X.reshape(train_X.shape[0],-1).T
train_Y_flatten=train_Y.reshape(train_Y.shape[0],-1).T

In [71]:
# Vì đây là ảnh màu nên phải scale image:
train_X_flatten=train_X_flatten/255
train_Y_flatten=train_Y_flatten/255

In [88]:
#Xây dựng các hàm bổ trợ để tính cost function
def sigmoid(x):
  return 1/(1+np.exp(-x))
def initialize(dim):
  w=np.zeros((dim,1))
  b=0
  assert(w.shape == (dim, 1))
  assert(isinstance(b, float) or isinstance(b, int))
  return w, b 
def propagate(w,b,X,Y):
  m=X.shape[1]
  A=sigmoid(np.dot(w.T,X)+b)
  cost=(-1/m)*np.sum((Y*np.log(A))+(1-Y)*np.log(1-A))
  dw=(1/m)*(np.dot(X,np.subtract(A,Y).T))
  db=(1/m)*(np.sum(A-Y))
  ### END CODE HERE ###
  assert(db.dtype == float)
    #assert(dw.shape == w.shape)
    
  cost = np.squeeze(cost)
  assert(cost.shape == ())

  grads = {"dw": dw,
            
            "db": db}
    
  return grads, cost
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
  costs=[]
  for i in range(num_iterations):
    grad,cost=propagate(w,b,X,Y)
    dw=grad["dw"]
    db=grad["db"]
    w=w-learning_rate*dw
    b=b-learning_rate*db
    if i % 100 == 0:
      costs.append(cost)
    if print_cost and i % 100 == 0:
      print ("Cost after iteration %i: %f" %(i, cost))
    
  params = {"w": w,
              "b": b}
    
  grads = {"dw": dw,
             "db": db}
    
  return params, grads, costs
def predict(w, b, X):
  m=X.shape[1]
  Y_predict=np.zeros((1,m))
  w=w.reshape(X.shape[0],1)
  A=sigmoid(np.dot(w.T,X)+b)
  for i in range (A.shape[1]):
    if A[0][i]<=0.5:
      Y_predict[0][i]=0
    else:
      Y_predict[0][i]=1
  assert(Y_predict.shape == (1, m))
    
  return Y_predict
def model(X_train, test_X,Y_train, test_Y, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
  w,b=initialize(X_train.shape[0])
  params, grads, costs=optimize(w, b, X_train, test_X, num_iterations, learning_rate, print_cost)
  w=params["w"]
  b=params["b"]
  Y_prediction_train=predict(w,b,X_train)
  Y_prediction_test=predict(w,b,Y_train)
  print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - test_X)) * 100))
  print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - test_Y)) * 100))

    
  d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
  return d


In [89]:
d = model(train_X_flatten, test_X, train_Y_flatten, test_Y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)


Cost after iteration 0: 0.693147
Cost after iteration 100: 0.584508
Cost after iteration 200: 0.466949
Cost after iteration 300: 0.376007
Cost after iteration 400: 0.331463
Cost after iteration 500: 0.303273
Cost after iteration 600: 0.279880
Cost after iteration 700: 0.260042
Cost after iteration 800: 0.242941
Cost after iteration 900: 0.228004
Cost after iteration 1000: 0.214820
Cost after iteration 1100: 0.203078
Cost after iteration 1200: 0.192544
Cost after iteration 1300: 0.183033
Cost after iteration 1400: 0.174399
Cost after iteration 1500: 0.166521
Cost after iteration 1600: 0.159305
Cost after iteration 1700: 0.152667
Cost after iteration 1800: 0.146542
Cost after iteration 1900: 0.140872
train accuracy: 99.04306220095694 %
test accuracy: 70.0 %
